# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [11]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact
import plotly.express as px

cf.go_offline()

In [180]:
data = pd.read_excel('../data/Online Retail.xlsx')
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [85]:
notUK = data[data["Country"] != "United Kingdom"]

In [86]:
ex1 = notUK[notUK["InvoiceDate"] > "2011-04-01 00:00:00"]
ex1_2 = ex1[month["InvoiceDate"] < "2011-04-30 23:59:59"]

In [87]:
ex1_2 = ex1_2.groupby("Country").sum().drop(["InvoiceNo", "UnitPrice", "CustomerID"], axis=1)

In [142]:
ex1_2.iplot(kind="bar", title="Total quantity and revenue by country month of April 2011")

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [135]:
france = data[data["Country"] == "France"][["Quantity", "Revenue", "InvoiceDate"]]

In [136]:
ex2_1 = france[france["InvoiceDate"] > "2011-01-01 00:00:00"]
ex2_2 = ex2_1[ex2_1["InvoiceDate"] < "2011-05-31 23:59:59"]

In [138]:
#Quiero que el diagrama esté ordenado por fecha.
ex2_2 = ex2_2.groupby("InvoiceDate").sum()
ex2_2.iplot(kind="line", title="France")

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [274]:
item = data[data["Description"] == "PARTY BUNTING"][["Quantity", "UnitPrice", "Country"]]

In [275]:
item = item.groupby("Country", as_index=False).mean()

In [277]:
item.iplot(x="Quantity", 
           y="UnitPrice", 
           categories="Country", 
           title="PARTY BUNTING Scatter plot")

"""import plotly.express as px
fig = px.scatter(item, x="Quantity", y="UnitPrice", color=item.index)
fig.show()"""

#Este gráfico no es correcto y soy incapaz de encontrar el error en la lógica.

'import plotly.express as px\nfig = px.scatter(item, x="Quantity", y="UnitPrice", color=item.index)\nfig.show()'

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [240]:
countriesdf = data[(data["Country"] == "EIRE") | 
                   (data["Country"] == "Germany") | 
                   (data["Country"] == "France") | 
                   (data["Country"] == "Netherlands")][["Quantity", "InvoiceNo", "Country"]]


In [241]:
countriesdf_pivot = countriesdf.pivot_table(values='Quantity', columns='Country', index='InvoiceNo', aggfunc='sum')

In [245]:
countriesdf_pivot.iplot(kind="hist", 
                        title="Distributions of quantity per invoice")

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [279]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [281]:
newCountriesdf = data[(data["Country"] == "EIRE") | 
                   (data["Country"] == "Germany") | 
                   (data["Country"] == "France") | 
                   (data["Country"] == "Netherlands")][["Revenue", "Description", "Country"]]

In [286]:
newCountriesdf2 = [e for e in product_list if newCountriesdf["Description"].str.contains(e, regex=False)]
    

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [288]:
if newCountriesdf["Description"].str.contains("JUMBO BAG RED RETROSPOT", regex=False):
    print("Sí")
else:
    print("No")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [289]:
newCountriesdf["Description"]

179        CREAM HANGING HEART T-LIGHT HOLDER
198        CREAM HANGING HEART T-LIGHT HOLDER
304        CREAM HANGING HEART T-LIGHT HOLDER
322        CREAM HANGING HEART T-LIGHT HOLDER
367        CREAM HANGING HEART T-LIGHT HOLDER
                         ...                 
396001    SET OF 6 RIBBONS PERFECTLY PRETTY  
396005    SET OF 6 RIBBONS PERFECTLY PRETTY  
396007    SET OF 6 RIBBONS PERFECTLY PRETTY  
396021                 SET OF 6 RIBBONS PARTY
396022                 SET OF 6 RIBBONS PARTY
Name: Description, Length: 26150, dtype: object

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [ ]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [ ]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.